In [1]:
import os

import numpy as np
import pandas as pd
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

from data import *


SEED = 662
DATA_DIR = "processed"

/Users/borisleung/anaconda3/envs/sta841kaggle/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
from itertools import combinations
from pyexpat import model
import traceback

import lightgbm as lgb
from sklearn.ensemble import StackingClassifier
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsTransformer
from sklearn.pipeline import Pipeline

from models import *


def load_objective(
    X: pd.DataFrame,
    y: pd.DataFrame,
    y_binarized: pd.DataFrame,
    get_trained_model: callable,
):
    X.replace(-999, np.nan, inplace=True)

    def objective(trial: optuna.Trial):
        chi2_threshold = trial.suggest_int("chi2_threshold", 2, 100)
        cv = 3
        imputer_strategy = trial.suggest_categorical(
            "imputer_strategy", ["mean", "median", "most_frequent"]
        )

        train_losses, valid_losses, valid_loss_shift, n_iters = [], [], [], []
        X_train, X_valid, y_train, y_valid, y_train_binarized, y_valid_binarized = (
            train_test_split(X, y, y_binarized, test_size=1 / cv, random_state=SEED)
        )  # lazy, just do 1 round of cv

        # compute pov means aggregated by psu
        aggregated_pov_train_data = pd.concat(
            [X_train["psu_hh_idcode"], y_train], axis=1
        )
        aggregated_pov_train_data[["psu", "hh", "idcode"]] = aggregated_pov_train_data[
            "psu_hh_idcode"
        ].str.split("_", expand=True)
        df_mean = aggregated_pov_train_data.groupby("psu")["num_pov"].mean()
        X_train_means = pd.merge(
            aggregated_pov_train_data,
            df_mean,
            on="psu",
            how="left",
            suffixes=("", "_mean"),
        )["num_pov_mean"]
        X_train_means.replace(np.nan, X_train_means.mean(), inplace=True)

        X_valid[["psu", "hh", "idcode"]] = X_valid["psu_hh_idcode"].str.split(
            "_", expand=True
        )
        X_valid_means = pd.merge(
            X_valid,
            df_mean,
            on="psu",
            how="left",
            suffixes=("", "_mean"),
        )["num_pov"]
        X_valid_means.replace(np.nan, X_valid_means.mean(), inplace=True)
        X_valid_means.rename("num_pov_mean", inplace=True)

        preprocessor = get_preprocessor(
            # ordinal_transformer=Pipeline([("imputer", "passthrough")]),
            imputer_strategy=[
                "most_frequent",
                "most_frequent",
                imputer_strategy,
                imputer_strategy,
            ],
            remainder="drop",
        )

        X_train_processed = preprocessor.fit_transform(X_train)
        selector = SelectPercentile(chi2, percentile=chi2_threshold)
        X_train = selector.fit_transform(X_train_processed, y_train)

        X_valid = preprocessor.transform(X_valid)
        X_valid = selector.transform(X_valid)

        # add pov means to the data
        X_train = np.column_stack([X_train, X_train_means])
        X_valid = np.column_stack([X_valid, X_valid_means])

        # add knn means too
        p = trial.suggest_int("p", 1, 3)
        k_neighbors = trial.suggest_int("k_neighbors", 2, 15)
        transformer = KNeighborsTransformer(
            mode="connectivity", n_neighbors=k_neighbors, p=p
        )
        X_dist_graph = transformer.fit_transform(X_train)
        X_dist_graph.setdiag(0)

        knn_train_means = np.divide(
            (X_dist_graph @ y_train.T), np.asarray(X_dist_graph.sum(axis=1)).flatten()
        )
        np.nan_to_num(knn_train_means, nan=np.nanmean(knn_train_means), copy=False)
        X_train = np.hstack([X_train, knn_train_means.reshape(-1, 1)])
        knn_valid_means = (
            (transformer.kneighbors_graph(X_valid) @ y_train.T) / k_neighbors
        ).reshape(-1, 1)
        np.nan_to_num(knn_valid_means, nan=np.nanmean(knn_train_means), copy=False)
        X_valid = np.hstack([X_valid, knn_valid_means])

        model = get_trained_model(
            trial, seed=SEED, X_train=X_train, y_train=y_train - 1
        )
        # train_losses.append(
        #     log_loss(y_train_binarized, model.predict_proba(X_train), normalize=False)
        #     / len(y_train_binarized)
        # )

        y_pred = model.predict_proba(X_valid)
        for column_index in [
            column for column in range(10) if column not in model.classes_
        ]:
            y_pred = np.insert(y_pred, column_index - 1, 0, axis=1)

        valid_losses.append(
            log_loss(y_valid_binarized, y_pred, normalize=False)
            / len(y_valid_binarized)
        )
        # valid_loss_shift.append(valid_losses[-1] - train_losses[-1])
        # n_iters.append(model.n_iter_[0])

        mean_valid_loss = np.mean(valid_losses)
        trial.set_user_attr(
            "selected_columns", X_train_processed.columns[selector.get_support()]
        )
        # trial.set_user_attr("n_iter", np.mean(n_iters))
        # trial.set_user_attr("train_loss", np.mean(train_losses))
        # trial.set_user_attr("valid_loss_shift", np.mean(valid_loss_shift))
        return mean_valid_loss

    return objective


models = {
    "naive_bayes": suggest_naive_bayes,
    "knn": suggest_knn_classifier,
    "lr": suggest_logistic_regression,
    "linear_svc": suggest_linear_svc,
    "kernel_svc": suggest_kernel_svc,
    "random_forest": suggest_random_forest,
    # "gradient_boosting": (get_gradient_boosting, 200),
    # "lightgbm": (get_lightgbm, 200),
    # "mlp": (get_mlp, 200),
}


optuna.logging.set_verbosity(optuna.logging.ERROR)
# data = get_divided_edu(
#     remove_boring_columns(transform_all_house(combined_transformed_train_with_num_pov))
# )
# poi = data[1]
data = remove_boring_columns(
    transform_education_levels(
        transform_all_house(combined_transformed_train_with_num_pov)
    )
)

poi = data
nunique = poi.nunique()
cols_to_drop = nunique[nunique == 1].index
poi = poi.drop(cols_to_drop, axis=1)
poi = poi.drop(columns=["edu_q09", "edu_q10", "edu_q11", "edu_q12", "edu_q13"])
print(poi.shape)
for choices in range(2, 4):
    for index, model_names in enumerate(combinations(models.keys(), choices)):
        best_values, best_params = [], []
        print(model_names)

        def get_trained_model(trial, seed, X_train, y_train):
            estimators = [
                (model_name, models[model_name](trial, max_iter=MAX_ITER, seed=seed))
                for model_name in model_names
            ]
            clf = StackingClassifier(
                estimators=estimators, final_estimator=LogisticRegression(max_iter=200)
            ).fit(X_train, y_train)
            return clf

        # if True:
        try:
            X, y, y_binarized = (
                poi.drop(Y_COLUMNS + ["num_pov"], axis=1),
                poi["num_pov"],
                poi[Y_COLUMNS],
            )
            study = optuna.create_study(direction="minimize")
            study.optimize(
                load_objective(X, y, y_binarized, get_trained_model),
                n_trials=100,
                n_jobs=-1,
                show_progress_bar=True,
            )
            best_values.append(study.best_value)
            best_params.append(study.best_params)
            print(optuna.importance.get_param_importances(study))
            print(study.best_trial.user_attrs)
        except Exception as e:
            print(traceback.format_exc())
        print(best_values), print(best_params)
        print("=====================================")
        print()
        # break

(5337, 83)
('naive_bayes', 'knn')


Best trial: 35. Best value: 1.77866: 100%|██████████| 100/100 [00:20<00:00,  4.81it/s]


{'chi2_threshold': 0.5227829752564205, 'n_neighbors': 0.3563907129207518, 'leaf_size': 0.053998449100407034, 'k_neighbors': 0.03883109228281512, 'p': 0.010077626862610694, 'algorithm': 0.00990479912274357, 'weights': 0.00477508354562651, 'imputer_strategy': 0.003239260908624691}
{'selected_columns': Index(['binary__house_q02_2.0', 'binary__house_q07_1.0',
       'binary__house_q07_2.0', 'binary__house_q11_2.0',
       'binary__house_q17_1.0', 'binary__edu_q03_2.0', 'binary__edu_q08_2.0',
       'binary__edu_q14_2.0', 'binary__edu_q25_2.0', 'binary__edu_q43_2.0',
       'binary__edu_q45_2.0', 'categorical__house_q03_2.0',
       'categorical__house_q03_infrequent_sklearn',
       'categorical__house_q06_4.0',
       'categorical__house_q06_infrequent_sklearn',
       'categorical__edu_q17_13.0', 'categorical__edu_q17_infrequent_sklearn',
       'categorical__edu_q23_infrequent_sklearn',
       'categorical__edu_q28_infrequent_sklearn',
       'categorical__edu_q42_infrequent_sklearn',
 

Best trial: 50. Best value: 1.74768: 100%|██████████| 100/100 [09:35<00:00,  5.76s/it]


{'chi2_threshold': 0.9734007031396735, 'l1_ratio': 0.01738177031154062, 'k_neighbors': 0.004793642015515371, 'imputer_strategy': 0.00167908533964254, 'C': 0.0015182236954004763, 'p': 0.0012265754982274848}
{'selected_columns': Index(['binary__house_q02_2.0', 'binary__house_q07_1.0',
       'binary__house_q07_2.0', 'binary__house_q11_2.0',
       'binary__house_q17_1.0', 'binary__edu_q03_2.0', 'binary__edu_q08_2.0',
       'binary__edu_q14_2.0', 'binary__edu_q25_2.0', 'binary__edu_q32_2.0',
       'binary__edu_q43_2.0', 'binary__edu_q45_2.0',
       'categorical__house_q03_2.0',
       'categorical__house_q03_infrequent_sklearn',
       'categorical__house_q06_4.0',
       'categorical__house_q06_infrequent_sklearn',
       'categorical__edu_q17_13.0', 'categorical__edu_q17_infrequent_sklearn',
       'categorical__edu_q23_infrequent_sklearn',
       'categorical__edu_q28_infrequent_sklearn',
       'categorical__edu_q42_infrequent_sklearn',
       'categorical__edu_q63_infrequent_sklea

...

{'chi2_threshold': 0.3867905427396493, 'l1_ratio': 0.15015548007610413, 'max_features': 0.1395106542766929, 'p': 0.07976860445903929, 'max_depth': 0.060344108764801306, 'criterion': 0.04296358829722028, 'min_samples_split': 0.04165521079738258, 'n_estimators': 0.03748283263220745, 'k_neighbors': 0.026828435063417082, 'imputer_strategy': 0.014926888577185627, 'min_samples_leaf': 0.011228564488089572, 'C': 0.0034535315896510803, 'penalty': 0.002847614294690611, 'intercept_scaling': 0.0012916978123532008, 'calibration_method': 0.0007522461315156685}
{'selected_columns': Index(['binary__house_q02_2.0', 'binary__house_q07_1.0',
       'binary__house_q07_2.0', 'binary__house_q11_2.0',
       'binary__house_q17_1.0', 'binary__edu_q03_2.0', 'binary__edu_q08_2.0',
       'binary__edu_q14_2.0', 'binary__edu_q25_2.0', 'binary__edu_q32_2.0',
       'binary__edu_q43_2.0', 'binary__edu_q45_2.0', 'binary__edu_q46_2.0',
       'binary__edu_q50_2.0', 'binary__edu_q57_2.0', 'binary__edu_q61_2.0',
      

Best trial: 75. Best value: 1.73629: 100%|██████████| 100/100 [14:50<00:00,  8.91s/it]


{'chi2_threshold': 0.7312636135061613, 'max_features': 0.08448780027764755, 'n_estimators': 0.0823318559615283, 'l1_ratio': 0.03418769844909908, 'min_samples_split': 0.018427104993198665, 'p': 0.014070495479315177, 'min_samples_leaf': 0.01193280456841838, 'kernel': 0.00726955829361717, 'max_depth': 0.005835158347943416, 'k_neighbors': 0.00432077670302759, 'C': 0.0027568237112476487, 'criterion': 0.002638527968500804, 'imputer_strategy': 0.0004777817402949007}
{'selected_columns': Index(['binary__house_q02_2.0', 'binary__house_q07_1.0',
       'binary__house_q07_2.0', 'binary__house_q11_2.0',
       'binary__house_q17_1.0', 'binary__edu_q03_2.0', 'binary__edu_q08_2.0',
       'binary__edu_q14_2.0', 'binary__edu_q25_2.0', 'binary__edu_q32_2.0',
       'binary__edu_q43_2.0', 'binary__edu_q45_2.0', 'binary__edu_q61_2.0',
       'binary__edu_q64_2.0', 'categorical__house_q03_2.0',
       'categorical__house_q03_infrequent_sklearn',
       'categorical__house_q06_4.0',
       'categorical__h

Best trial: 83. Best value: 1.7342: 100%|██████████| 100/100 [05:53<00:00,  3.53s/it]


{'chi2_threshold': 0.5662645955414046, 'max_features': 0.201228677632925, 'n_estimators': 0.13354274200014088, 'imputer_strategy': 0.03392393074061137, 'min_samples_split': 0.022470173526911413, 'kernel': 0.015053049711494688, 'k_neighbors': 0.008437995821772622, 'min_samples_leaf': 0.006492635886290935, 'max_depth': 0.005683983694044486, 'penalty': 0.003013942096120324, 'p': 0.0015791540231456958, 'criterion': 0.0012303750248893078, 'C': 0.0004740825343334017, 'calibration_method': 0.00039822329360978065, 'intercept_scaling': 0.00020643847230534775}
{'selected_columns': Index(['binary__house_q02_2.0', 'binary__house_q07_1.0',
       'binary__house_q07_2.0', 'binary__house_q11_2.0',
       'binary__house_q17_1.0', 'binary__edu_q03_2.0', 'binary__edu_q08_2.0',
       'binary__edu_q14_2.0', 'binary__edu_q43_2.0',
       'categorical__house_q03_2.0',
       'categorical__house_q03_infrequent_sklearn',
       'categorical__house_q06_4.0',
       'categorical__house_q06_infrequent_sklearn',